In [1]:
import pandas as pd
import requests
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [3]:



API_KEY = "8edf5a3557214b83a8d24412250109"
local = "Curitiba"  # pode ser "Curitiba, Brazil" ou latitude,longitude
url = "http://api.weatherapi.com/v1/forecast.json"
params = {
    "key": API_KEY,
    "q": local,
    "lang": "pt",     # opcional: traduz descrições para português
    "aqi": "no"       # opcional: inclui dados de qualidade do ar
}

try:
    response = requests.get(url, params=params)
    response.raise_for_status()
    dados = response.json()
    
    # Exemplo de como acessar alguns dados:
    # temperatura = dados["current"]["temp_c"]
    # descricao = dados["current"]["condition"]["text"]
    # umidade = dados["current"]["humidity"]
    # vento = dados["current"]["wind_kph"]

    # print(f"Tempo em {dados['location']['name']}: {descricao.capitalize()}")
    # print(f"Temperatura: {temperatura} °C")
    # print(f"Umidade: {umidade}%")
    # print(f"Vento: {vento} km/h")
   # print(dados)

except requests.exceptions.HTTPError as err:
    print(f"Erro HTTP: {err} — código: {response.status_code}")
except ValueError:
    print("Erro ao decodificar JSON.")
except Exception as e:
    print(f"Ocorreu um erro: {e}")

df = pd.json_normalize(dados)
df

    


,location.name,location.region,location.country,location.lat,location.lon,location.tz_id,location.localtime_epoch,location.localtime,current.last_updated_epoch,current.last_updated,current.temp_c,current.temp_f,current.is_day,current.condition.text,current.condition.icon,current.condition.code,current.wind_mph,current.wind_kph,current.wind_degree,current.wind_dir,current.pressure_mb,current.pressure_in,current.precip_mm,current.precip_in,current.humidity,current.cloud,current.feelslike_c,current.feelslike_f,current.windchill_c,current.windchill_f,current.heatindex_c,current.heatindex_f,current.dewpoint_c,current.dewpoint_f,current.vis_km,current.vis_miles,current.uv,current.gust_mph,current.gust_kph,forecast.forecastday
0,Curitiba,Parana,Brazil,-25.4167,-49.25,America/Sao_Paulo,1758320317,2025-09-19 19:18,1758320100,2025-09-19 19:15,20.0,68.0,0,Céu limpo,//cdn.weatherapi.com/weather/64x64/night/113.png,1000,3.8,6.1,87,E,1018.0,30.06,0.0,0.0,78,0,20.0,68.0,17.5,63.6,17.5,63.6,12.7,54.8,10.0,6.0,0.0,7.9,12.7,"[{'date': '2025-09-19', 'date_epoch': 17582400..."


In [4]:
forecast_list = dados["forecast"]["forecastday"]
df_forecast = pd.json_normalize(forecast_list)
df_forecast


,date,date_epoch,hour,day.maxtemp_c,day.maxtemp_f,day.mintemp_c,day.mintemp_f,day.avgtemp_c,day.avgtemp_f,day.maxwind_mph,day.maxwind_kph,day.totalprecip_mm,day.totalprecip_in,day.totalsnow_cm,day.avgvis_km,day.avgvis_miles,day.avghumidity,day.daily_will_it_rain,day.daily_chance_of_rain,day.daily_will_it_snow,day.daily_chance_of_snow,day.condition.text,day.condition.icon,day.condition.code,day.uv,astro.sunrise,astro.sunset,astro.moonrise,astro.moonset,astro.moon_phase,astro.moon_illumination,astro.is_moon_up,astro.is_sun_up
0,2025-09-19,1758240000,"[{'time_epoch': 1758250800, 'time': '2025-09-1...",27.6,81.6,11.9,53.3,19.0,66.2,5.1,8.3,0.0,0.0,0.0,9.3,5.0,61,0,0,0,0,Sol,//cdn.weatherapi.com/weather/64x64/day/113.png,1000,2.3,06:09 AM,06:12 PM,04:53 AM,04:23 PM,Waning Crescent,8,0,0


In [5]:
hour_list = dados["forecast"]["forecastday"][0]["hour"]
df_hour = pd.json_normalize(hour_list)
df_hour


,time_epoch,time,temp_c,temp_f,is_day,wind_mph,wind_kph,wind_degree,wind_dir,pressure_mb,pressure_in,precip_mm,precip_in,snow_cm,humidity,cloud,feelslike_c,feelslike_f,windchill_c,windchill_f,heatindex_c,heatindex_f,dewpoint_c,dewpoint_f,will_it_rain,chance_of_rain,will_it_snow,chance_of_snow,vis_km,vis_miles,gust_mph,gust_kph,uv,condition.text,condition.icon,condition.code
0,1758250800,2025-09-19 00:00,14.1,57.3,0,5.1,8.3,79,E,1020.0,30.11,0.0,0.0,0.0,85,57,13.8,56.8,13.8,56.8,14.1,57.3,11.5,52.7,0,0,0,0,10.0,6.0,6.7,10.8,0.0,Parcialmente nublado,//cdn.weatherapi.com/weather/64x64/night/116.png,1003
1,1758254400,2025-09-19 01:00,14.3,57.7,0,5.1,8.3,77,ENE,1019.0,30.09,0.0,0.0,0.0,82,55,14.0,57.2,14.0,57.2,14.3,57.7,11.2,52.2,0,0,0,0,10.0,6.0,6.7,10.8,0.0,Parcialmente nublado,//cdn.weatherapi.com/weather/64x64/night/116.png,1003
2,1758258000,2025-09-19 02:00,14.3,57.7,0,4.7,7.6,72,ENE,1018.0,30.06,0.0,0.0,0.0,81,24,14.1,57.4,14.1,57.4,14.3,57.7,11.1,52.0,0,0,0,0,10.0,6.0,6.2,9.9,0.0,Céu limpo,//cdn.weatherapi.com/weather/64x64/night/113.png,1000
3,1758261600,2025-09-19 03:00,13.0,55.5,0,2.5,4.0,89,E,1018.0,30.06,0.0,0.0,0.0,89,40,13.5,56.3,13.5,56.3,13.0,55.5,11.3,52.3,0,0,0,0,10.0,6.0,3.7,6.0,0.0,Parcialmente nublado,//cdn.weatherapi.com/weather/64x64/night/116.png,1003
4,1758265200,2025-09-19 04:00,12.0,53.5,0,1.8,2.9,105,ESE,1018.0,30.07,0.0,0.0,0.0,94,54,12.0,53.5,12.0,53.5,12.0,53.5,11.0,51.9,0,0,0,0,2.0,1.0,3.1,5.0,0.0,Neblina,//cdn.weatherapi.com/weather/64x64/night/143.png,1030
5,1758268800,2025-09-19 05:00,11.9,53.3,0,0.9,1.4,94,E,1019.0,30.09,0.0,0.0,0.0,95,56,11.9,53.3,11.9,53.3,11.9,53.3,11.0,51.8,0,0,0,0,2.0,1.0,1.7,2.7,0.0,Neblina,//cdn.weatherapi.com/weather/64x64/night/143.png,1030
6,1758272400,2025-09-19 06:00,12.9,55.3,0,2.0,3.2,74,ENE,1019.0,30.10,0.0,0.0,0.0,90,37,13.6,56.5,13.6,56.5,12.9,55.3,11.3,52.3,0,0,0,0,10.0,6.0,3.3,5.3,0.0,Parcialmente nublado,//cdn.weatherapi.com/weather/64x64/night/116.png,1003
7,1758276000,2025-09-19 07:00,15.6,60.0,1,2.5,4.0,68,ENE,1020.0,30.11,0.0,0.0,0.0,76,9,15.6,60.0,15.6,60.0,15.6,60.0,11.4,52.4,0,0,0,0,10.0,6.0,2.9,4.6,0.2,Sol,//cdn.weatherapi.com/weather/64x64/day/113.png,1000
8,1758279600,2025-09-19 08:00,18.5,65.4,1,3.1,5.0,46,NE,1019.0,30.10,0.0,0.0,0.0,61,35,18.5,65.4,18.5,65.4,18.5,65.4,11.0,51.7,0,0,0,0,10.0,6.0,3.6,5.8,1.2,Parcialmente nublado,//cdn.weatherapi.com/weather/64x64/day/116.png,1003
9,1758283200,2025-09-19 09:00,21.5,70.7,1,3.1,5.0,30,NNE,1019.0,30.08,0.0,0.0,0.0,47,82,21.5,70.7,21.5,70.7,24.2,75.6,9.6,49.2,0,0,0,0,10.0,6.0,3.6,5.8,3.2,Nublado,//cdn.weatherapi.com/weather/64x64/day/119.png,1006


In [6]:
# Lista apenas com as colunas importantes
colunas_importantes = [
    "time",
    "feelslike_c",
    "humidity",
    "cloud",
    "uv",
    "wind_kph",
    "will_it_rain",
    "chance_of_rain",
    "condition.text"
]

df_limpo = df_hour[colunas_importantes]
df_limpo.sample(5)

,time,feelslike_c,humidity,cloud,uv,wind_kph,will_it_rain,chance_of_rain,condition.text
2,2025-09-19 02:00,14.1,81,24,0.0,7.6,0,0,Céu limpo
5,2025-09-19 05:00,11.9,95,56,0.0,1.4,0,0,Neblina
10,2025-09-19 10:00,24.5,36,4,5.7,4.3,0,0,Sol
15,2025-09-19 15:00,25.9,25,0,5.1,7.6,0,0,Sol
16,2025-09-19 16:00,25.3,29,0,2.4,7.9,0,0,Sol


In [7]:
# Cria uma tabela associando estilos de roupa com dados do clima
df_tabela1 = pd.DataFrame({
    "estilos": ["casual", "esportivo", "social", "streetwear"],
    "data e hora": df_limpo["time"].values[:4],
    "temperatura": df_limpo["feelslike_c"].values[:4],
    "chance de chuva": df_limpo["chance_of_rain"].values[:4],
    "umidade": df_limpo["humidity"].values[:4],
    "uv": df_limpo["uv"].values[:4],
    "condição do dia": df_limpo ["condition.text"].values[:4]
})

df_tabela1

,estilos,data e hora,temperatura,chance de chuva,umidade,uv,condição do dia
0,casual,2025-09-19 00:00,13.8,0,85,0.0,Parcialmente nublado
1,esportivo,2025-09-19 01:00,14.0,0,82,0.0,Parcialmente nublado
2,social,2025-09-19 02:00,14.1,0,81,0.0,Céu limpo
3,streetwear,2025-09-19 03:00,13.5,0,89,0.0,Parcialmente nublado


In [8]:
print(df_tabela1.columns.tolist())

['estilos', 'data e hora', 'temperatura', 'chance de chuva', 'umidade', 'uv', 'condição do dia']


In [9]:
#transformando variáveis e em um número categorico 

x = pd.get_dummies(df_tabela1.drop(columns=["estilos"]))
y = df_tabela1["estilos"]

In [10]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
clf = DecisionTreeClassifier(max_depth=4, random_state=42)
clf.fit(X_train, y_train)


,criterion,'gini'
,splitter,'best'
,max_depth,4
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,42
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,class_weight,None


In [11]:
# --- Previsão automática para todas as linhas da tabela ---
# Criar dummies para toda a tabela e alinhar com colunas do treino
X_dummies = pd.get_dummies(df_tabela1.drop(columns=["estilos"]))
X_dummies = X_dummies.reindex(columns=x.columns, fill_value=0)


In [12]:
df_tabela1["predicao_estilo"] = clf.predict(X_dummies)

print(df_tabela1)



      estilos       data e hora  temperatura  chance de chuva  umidade   uv  \
0      casual  2025-09-19 00:00         13.8                0       85  0.0   
1   esportivo  2025-09-19 01:00         14.0                0       82  0.0   
2      social  2025-09-19 02:00         14.1                0       81  0.0   
3  streetwear  2025-09-19 03:00         13.5                0       89  0.0   

        condição do dia predicao_estilo  
0  Parcialmente nublado          casual  
1  Parcialmente nublado          casual  
2             Céu limpo          social  
3  Parcialmente nublado      streetwear  
